In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from flows import Invert
from flows.glow import GlowFlow, coupling_nn_glow
from models import FlowLVM
from models.optimization import LinearWarmupSchedule
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Input, Concatenate
from tensorflow.keras import Model
#tf.config.experimental_run_functions_eagerly(True)

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
batch_size = 20
X_train = np.expand_dims(X_train, axis=-1)
X_train_ds = tf.data.Dataset.from_tensor_slices(X_train.astype(np.float32))
X_train_ds = X_train_ds.map(lambda x: x / 255. - 0.5).shuffle(buffer_size=X_train.shape[0]).batch(batch_size)

In [ ]:
glow = Invert(GlowFlow(num_layers=3, depth=8, coupling_nn_ctor=coupling_nn_glow(hidden_dims=64)))
learning_rate = LinearWarmupSchedule(1.0E-3, num_warmup_steps=100)
#learning_rate = tf.keras.optimizers.schedules.InverseTimeDecay(1.0E-1, N_train//sample_batch_size, 0.1)
model = FlowLVM(glow, input_shape=(None,*X_train.shape[1:]), num_bins=255,
                optimizer=tf.keras.optimizers.Adamax(learning_rate=learning_rate))

In [ ]:
model.train(X_train_ds, X_train.shape[0] // batch_size, num_epochs=2)

In [ ]:
samples = model.sample(n=6)

In [ ]:
import matplotlib.pyplot as plt
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(np.squeeze(255*(samples[i]+0.5)))
plt.show()